In [2]:
!pip install av miditok py_midicsv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 42.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.9 MB/s eta 0:00:00:00:01


In [3]:
import av
import numpy as np
import random

from transformers import VivitImageProcessor, VivitModel, VivitConfig, TransfoXLLMHeadModel, TransfoXLConfig
from huggingface_hub import hf_hub_download
# print(transformers.__version__)
import matplotlib.pyplot as plt
from torch import nn
import py_midicsv as pm
import torch
import torch.nn.functional as F

from PIL import Image
import os
from miditok import REMI, TokenizerConfig  # here we choose to use REMI
import miditok
from pathlib import Path
from torch.cuda.amp import autocast

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data.dataloader import default_collate
from torch import optim

from tqdm import tqdm
import wandb

2024-05-17 14:34:09.254294: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 14:34:09.254394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 14:34:09.373566: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# elle
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

def cfg_to_dict(cfg):
    return {attr: getattr(cfg, attr) for attr in dir(cfg) if not attr.startswith("__") and not callable(getattr(cfg, attr))}

def read_frames_from_path(frames_path, indices, rgb=False):
    '''
    Read specific frames from a directory containing image files of video frames.
    Args:
        frames_path (str): Path to the directory containing frame images.
        indices (List[int]): List of frame indices to read.
    Returns:
        result (np.ndarray): numpy array of frames of shape (num_frames, height, width, 3).
    '''
    # List all files in the directory and sort them to maintain order
    all_files = sorted(os.listdir(frames_path))
    frames = []

    # Process only files at specific indices
    color_mode = 'RGB' if rgb else 'L'
    for idx in indices:
        if idx < len(all_files):
            file_path = os.path.join(frames_path, all_files[idx])
            with Image.open(file_path) as img:
                # Convert image to RGB to ensure consistency
                img = img.convert(color_mode)
                # Calculate differences to make the image square
                width, height = img.size
                max_side = max(width, height)
                # Create a new image with a black background
                new_img = Image.new(color_mode, (max_side, max_side))
                # Paste the original image onto the center of the new image
                new_img.paste(img, ((max_side - width) // 2, (max_side - height) // 2))
                frame_array = np.array(new_img)
                if color_mode == 'L':
                    # Expand dims to add the channel dimension, resulting in (H, W, 1)
                    frame_array = np.expand_dims(frame_array, axis=-1)
                frames.append(frame_array)

    stacked_frames = np.stack(frames, axis=0)
    return stacked_frames

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

In [8]:
class Video2MIDIDataset(Dataset):
    def __init__(self, root_dir, tokenizer, image_processor, transform=None, color_mode='gray'):
        self.root_dir = root_dir
        self.frames_dir = os.path.join(root_dir, 'frames')
        self.midi_dir = os.path.join(root_dir, 'midi')
        self.tokenizer = tokenizer
        self.image_processor = image_processor
        self.transform = transform
        self.piece_names = [d for d in os.listdir(self.frames_dir) if os.path.isdir(os.path.join(self.frames_dir, d))]
#         self.piece_names = self.piece_names[:2] # TODO: remove!!!
        assert self.piece_names, f"frame_dir at {self.frames_dir} is empty!"
        self.color_mode = 'L' if color_mode == 'gray' else 'RGB'

    def __len__(self):
        return len(self.piece_names)

    def __getitem__(self, idx):
        piece_name = self.piece_names[idx]
        frames_path = os.path.join(self.frames_dir, piece_name)
        midi_path = os.path.join(self.midi_dir, f'{piece_name}.mid')

        midi_token_ids = self.load_midi(midi_path)
        frames = self.load_frames(frames_path, rgb=True)
#         img_side_width = 64
#         img_size = (img_side_width,img_side_width)
#         print(frames.shape, type(frames))
        # NOTE (elle): MUST convert frames to list for some reason otherwise it complains!!!
#         processed_frames = self.image_processor(list(frames), return_tensors="pt", do_center_crop=False, do_resize=True, size=img_size)
        processed_frames = image_processor(list(frames), return_tensors="pt")
#         print(processed_frames)

        sample = {'frames': processed_frames['pixel_values'], 'midi_tokens': midi_token_ids}
        return sample

#     def load_frames(self, frames_path):
#         frame_files = sorted(os.listdir(frames_path))
#         frames = [Image.open(os.path.join(frames_path, f)).convert(self.color_mode) for f in frame_files]
#         print(f"img shape: {print(frames[0].size)}")
#         if self.transform:
#             frames = [self.transform(frame) for frame in frames]
#         return frames

    def load_frames(self, frames_path, rgb=False):
        '''
        Read specific frames from a directory containing image files of video frames.
        Args:
            frames_path (str): Path to the directory containing frame images.
            indices (List[int]): List of frame indices to read.
        Returns:
            result (np.ndarray): numpy array of frames of shape (num_frames, height, width, 3).
        '''
        # List all files in the directory and sort them to maintain order
        frame_names = sorted(os.listdir(frames_path))
        frames = []

        # Process only files at specific indices
        color_mode = 'RGB' if rgb else 'L'
        for frame_name in frame_names:
            file_path = f"{frames_path}/{frame_name}"
            with Image.open(file_path) as img:
                # Convert image to RGB to ensure consistency
                img = img.convert(color_mode)
                # Calculate differences to make the image square
                width, height = img.size
                max_side = max(width, height)
                # Create a new image with a black background
                new_img = Image.new(color_mode, (max_side, max_side))
                # Paste the original image onto the center of the new image
                new_img.paste(img, ((max_side - width) // 2, (max_side - height) // 2))
                frame_array = np.array(new_img)
                if color_mode == 'L':
                    # Expand dims to add the channel dimension, resulting in (H, W, 1)
                    frame_array = np.expand_dims(frame_array, axis=-1)
                frames.append(frame_array)

        stacked_frames = np.stack(frames, axis=0)
        return stacked_frames

    def load_midi(self, midi_path):
        midi_tokens = self.tokenizer(midi_path)
        midi_token_ids = torch.tensor(midi_tokens[0].ids, dtype=torch.long)
        return midi_token_ids

def custom_collate_fn(batch, tokenizer):
    # Extract frames and midi_tokens from the batch
    frames = [item['frames'] for item in batch]
    midi_tokens = [item['midi_tokens'] for item in batch]

    # Pad the midi_tokens
    # Assuming tokenizer provides PAD token index via tokenizer['PAD_None']
    pad_token_index = tokenizer['PAD_None']  # Ensure this is the correct index for your PAD token
    # print length before padding
#     print("Length before padding: ", [len(midi_token) for midi_token in midi_tokens])
    midi_tokens_padded = pad_sequence(midi_tokens, batch_first=True, padding_value=pad_token_index)
#     print("Length after padding: ", [len(midi_token) for midi_token in midi_tokens_padded])

    # Collate frames normally (assuming they are tensors of the same shape)
    frames = default_collate(frames)

    # Return a new dictionary with padded midi_tokens and frames
    return {'frames': frames, 'midi_tokens': midi_tokens_padded}

def show_images_and_midi(dataloader):
    for i, batch in enumerate(dataloader):
        frames = batch['frames']  # Assuming frames are tensors of shape (batch_size, channels, extra_dim, another_channel_like, height, width)
        midi_tokens = batch['midi_tokens']  # MIDI tokens

        print(f"Batch {i + 1}")

        # Calculate the number of rows and columns for the subplots
        batch_size = frames.size(0) # torch.Size([4, 1, 302, 1, 64, 64])
#         n_igms_in_batch = frames.size(2)
        cols = int(np.ceil(np.sqrt(batch_size)))
        rows = int(np.ceil(batch_size / cols))

        # Displaying images in a grid that's as square as possible
        fig, axs = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))  # Adjust size as needed
        axs = axs.flatten()  # Flatten the array of axes to make indexing easier

        for j in range(batch_size):
            # Select the first image from the video sequence and remove the singleton dimensions
            img = frames[j, 0, 0, 0]  # Reduces to (64, 64)
            axs[j].imshow(img.numpy())
            axs[j].axis('off')  # Hide axes
            axs[j].set_title(f'MIDI: {midi_tokens[j]}')  # Optionally print MIDI token IDs

        # Hide any unused axes if the total number of subplots exceeds the batch size
        for k in range(batch_size, len(axs)):
            axs[k].axis('off')

        plt.show()

        # Optional: stop after first batch for demonstration
        if i == 0:
            break

def show_frames(frames):
    print("First Batch")

    # Calculate the total number of images to display
    total_images = len(frames)  # Assuming 'extra_dim' holds 32 images

    cols = int(np.ceil(np.sqrt(total_images)))
    rows = int(np.ceil(total_images / cols))

    # Displaying images in a grid that's as square as possible
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))  # Adjust size as needed
    axs = axs.flatten()  # Flatten the array of axes to make indexing easier

    for j in range(total_images):
        # Select the image from the video sequence for each frame in the batch
        img = frames[j]  # Adjust indexing based on your data's shape, using the first item in batch
        axs[j].imshow(img.numpy())
        axs[j].axis('off')  # Hide axes
        axs[j].set_title(f'MIDI: {midi_tokens[0]}')  # Optionally print MIDI token IDs for the first item in batch

    # Hide any unused axes if the total number of subplots exceeds the total images
    for k in range(total_images, len(axs)):
        axs[k].axis('off')

    plt.show()

def show_images_and_midi_one_batch(dataloader, tokenizer):
    # Fetch the first batch from the dataloader
    token_id_to_token = {v: k for k, v in tokenizer.vocab.items()}
    batch = nextclas(iter(dataloader))
    batch_i = 0
    frames = batch['frames']  # Assuming frames are tensors of shape (batch_size, channels, extra_dim, another_channel_like, height, width)
    # ^ [4, 1, 32, 3, 224, 224]
    midi_tokens = batch['midi_tokens']  # MIDI tokens
    print(f"miditokens shape {midi_tokens.shape}")
    print("First Batch")

    # Calculate the total number of images to display
    total_images = frames.size(2)  # Assuming 'extra_dim' holds 32 images

    cols = int(np.ceil(np.sqrt(total_images)))
    rows = int(np.ceil(total_images / cols))

    # Displaying images in a grid that's as square as possible
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))  # Adjust size as needed
    axs = axs.flatten()  # Flatten the array of axes to make indexing easier

    midi_translations = []
    midis = []
    tokenss = []
    for j in range(total_images):
        # Select the image from the video sequence for each frame in the batch
        img = frames[0, 0, j] # Adjust indexing based on your data's shape, using the first item in batch
        axs[j].imshow(img.permute(1, 2, 0).numpy())
        axs[j].axis('off')  # Hide axes
        tokens = midi_tokens[batch_i]
        tokenss.append(tokens)
        midi = tokenizer([tokens])
        midis.append(midi)
        midi_translation = [token_id_to_token[int(id_.detach().numpy())] for id_ in tokens]
        midi_translations.append(midi_translation)
#         axs[j].set_title(f'MIDI: {midi_tokens_translated}')  # Optionally print MIDI token IDs for the first item in batch

    # Hide any unused axes if the total number of subplots exceeds the total images
    for k in range(total_images, len(axs)):
        axs[k].axis('off')

    title = f"{midis[batch_i]}\n{midi_translations[batch_i]}\n{tokenss[batch_i]}"
    fig.suptitle(title, fontsize=16)
    print(title)

    plt.show()

# dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)
def collate_fn(batch):
    return custom_collate_fn(batch, tokenizer)

def compute_accuracy(outputs, labels):
    logits = outputs.logits
    prediction_ids = torch.argmax(logits, dim=-1)
    # Flatten the tensors to compare each token
    prediction_ids = prediction_ids.view(-1)
    labels = labels.view(-1)
    
    # Compare predictions with labels
    correct = (prediction_ids == labels).sum().item()
    total = labels.size(0)

    accuracy = correct / total
    return accuracy, correct, total

# TODO
* investigate why this training loop (using real data but just twinkle sequence 0, one second) seems to work now -- is it AdamW, the LR, or probably just that getting a transformer to overfit on random labels won't work because not properly padded with start/end token? although idk bc I feel like I tried it with non-random at first and it didn't work so truly I have no idea
* why does the loss not quite go to 0 though? It hovers around .03 and then is a bit unstable and goes to .3 even sometimes e.g. a continuation of it although btw here https://wandb.ai/elles/video2music/runs/kmfa7au7?nw=nwuserellesummer I would think it should go to .0005ish which is what happened when I overfit to a single label sequence of the same numbers although at one point it did get to 0.0086 (not tracked in wandb). Just feel like it should be trivially easy/stable to memorize one one-sec sequence idk
* BUT good news: despite loss not being ~0, the output of the model is still the given sequence, so all is good heh
* ^^ take it further and track wer metric in general AH or just the accuracy tbh like per token (if it doesn't match for example)

btw for original model/dataset (not ours! aka the audio one)
shapes:
labels: torch.Size([1, 67]). encoder_outputs: torch.Size([1, 112, 768]). labels: torch.Size([1, 22]). encoder_outputs: torch.Size([1, 113, 768]).

our model tho is like: encoder_outputs: torch.Size([1, 3137, 768]) labels: torch.Size([1, 16])  
with 2 batch size  
label shape torch.Size([2, 20])  
frame shape torch.Size([2, 33, 3, 224, 224])  
encoder_outputs shape torch.Size([2, 3137, 768])  

## NEXT
* last progress: able to fully overfit on twinkle 1 second clips but really slowly (converged to 100% accuracy after an hour yikes)
* is slowness bc of the size of the encoder? is this worth the trade off of not having to train our own encoder? or is it bc the image quality has a lot of noise when resizing to fit the encoder needs? maybe worth using Khalil's modified encoder then...
* will the model scale well to e.g. 2-5 second clips? (dimitrios suggested 5 sec) and across different songs?
* try creating validation set just within twinkle twinkle? how to create a good one for this task in general? e.g. what should be "allowed" to overlap? We should see much less overlap in both inp/output when using longer sequences e.g. due to permutation diversity
* debug why the original training didn't work for your own sanity/learning

In [12]:
from transformers import SpeechT5ForSpeechToText, SpeechT5Config
from datasets import load_metric

ds_dir = "../input/data-32fps/data_32fps"
# frames_path = f"{ds_dir}/processed/frames/somebody--twinke_twinkle_0"
# frames_path = f"{ds_dir}/processed/frames/raw--twinke_twinkle_0"
# Our parameters
TOKENIZER_PARAMS = {
    "pitch_range": (21, 109),
    "beat_res": {(0, 4): 8, (4, 12): 4}, # TODO: finetune beat res?
    "num_velocities": 32,
    "special_tokens": ["PAD", "BOS", "EOS", "MASK"],
    "use_chords": True,
    "use_rests": False,
    "use_tempos": True,
    "use_time_signatures": False,
    "use_programs": False,
    "num_tempos": 32,  # number of tempo bins
    "tempo_range": (40, 250),  # (min, max)
}
tok_config = TokenizerConfig(**TOKENIZER_PARAMS)
# wer_metric = load_metric("wer")
# Creates the tokenizer
tokenizer = REMI(tok_config)
image_processor = VivitImageProcessor.from_pretrained("google/vivit-b-16x2-kinetics400")
dataset = Video2MIDIDataset(
    root_dir=f"{ds_dir}/processed",
    tokenizer=tokenizer,
    image_processor=image_processor
    # transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
)
print(f"Training on {len(dataset.piece_names)} pieces: {dataset.piece_names}")

config = SpeechT5Config(
    vocab_size=328,
    d_model=768,
    max_length=450
)

model = SpeechT5ForSpeechToText(config)
model = model.to('cuda')  # Ensure your model is on GPU if available
model_enc = VivitModel.from_pretrained("google/vivit-b-16x2-kinetics400")
model_enc = model_enc.to('cuda')
model.speecht5.encoder = None

class Config:
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    SEED = 43
    
    LOG_EVERY_X_EPOCHS = 1

    LR = 0.0001 #5e-5 # 4.786300923226385e-05 # 0.0001
    EPOCHS = 100
    BATCH_SIZE = 5

    USE_WANDB = True
    WANDB_PROJECT = "video2music"
    WANDB_ENTITY = "elles"
    WANDB_GROUP = f"full_twinkle_bsz{BATCH_SIZE}_lr{LR}"
    EXPERIMENT = WANDB_GROUP
    # FPS = 32

cfg = Config()

if cfg.SEED:
    set_seed(cfg.SEED)

optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.LR)
# Training loop
model.train()

if cfg.USE_WANDB:
    # convert cfg class to dict
    cfg_dict = cfg_to_dict(cfg)
    cfg_dict.update(config.to_dict())
    print(cfg_dict)
    assert cfg_dict != {}, "cfg_dict is empty"
    wandb_id = wandb.util.generate_id()
    wandb.init(
        project=cfg.WANDB_PROJECT,
        name=cfg.EXPERIMENT,
        entity=cfg.WANDB_ENTITY,
        config=cfg_dict,
        id=wandb_id,
        resume="allow",
        group=cfg.WANDB_GROUP
    )
    wandb.watch(model, log="all", log_freq=10)

epochs = cfg.EPOCHS

for epoch in range(epochs):
    total_correct = 0
    total_tokens = 0
    total_loss = 0
    progress_bar = tqdm(dataloader, desc=f'Epoch {epoch + 1}/{epochs}')
    model.train()
    for batch in progress_bar:
  
        optimizer.zero_grad()
        frames = batch['frames'].to('cuda')
        labels = batch['midi_tokens'].to('cuda')
        with torch.no_grad():
#             frames = batch['frames']

            # labels = rand_labels.unsqueeze(0)
#             labels = batch['midi_tokens']
#             frames = frames.to('cuda')
            # squeeze only if batch-size is > 1
            frames = frames.squeeze(1)
#             labels = labels.to('cuda')
            outputs = model_enc(frames)
            last_hidden_states = outputs.last_hidden_state
#         new_encoder_outputs = last_hidden_states.to('cuda')
        new_encoder_outputs = (last_hidden_states,)
        inputs = {
            "encoder_outputs": new_encoder_outputs,
            "labels": labels # labels.unsqueeze(0) only unsqueeze if batch size is 1?
        }
        outputs = model(**inputs)

        # Loss computation
        loss = outputs.loss
        total_loss += loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()

        
        progress_bar.set_postfix({'loss': loss.item()})

    # Logging to wandb
    average_loss = total_loss / len(dataloader)
    if epoch % cfg.LOG_EVERY_X_EPOCHS == 0:
        # TODO: add eval wer-metric
        model.eval()
        # TODO: iterate over training dataloader or eval dataloader
        # because right now we're just using last batch output
        with torch.no_grad():
            inputs = {
                "encoder_outputs": new_encoder_outputs,
                "labels": labels #labels.unsqueeze(0) only unsqueeze if batch size is 1?
            }
            outputs = model(**inputs)
            accuracy, batch_corr, batch_tot = compute_accuracy(outputs, labels)

            # Accumulate results
            # TODO fix: this is not accumulating per batch..
            total_correct += batch_corr
            total_tokens += batch_tot

            overall_accuracy = total_correct / total_tokens
            stats = {'epoch': epoch, 'epoch_avg_loss': average_loss, 'batch_accuracy': overall_accuracy}
            print(stats)
            if wandb.run:
                wandb.log(stats)
    del frames, labels, outputs, last_hidden_states, new_encoder_outputs, inputs
    torch.cuda.empty_cache()

# End of training
if wandb.run:
  wandb.finish()

/tmp/ipykernel_34/872650869.py:21: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  tok_config = TokenizerConfig(**TOKENIZER_PARAMS)


Training on 29 pieces: ['raw--twinke_twinkle_0', 'raw--twinke_twinkle_1', 'raw--twinke_twinkle_11', 'raw--twinke_twinkle_16', 'raw--twinke_twinkle_24', 'raw--twinke_twinkle_17', 'raw--twinke_twinkle_28', 'raw--twinke_twinkle_6', 'raw--twinke_twinkle_5', 'raw--twinke_twinkle_10', 'raw--twinke_twinkle_4', 'raw--twinke_twinkle_26', 'raw--twinke_twinkle_7', 'raw--twinke_twinkle_19', 'raw--twinke_twinkle_8', 'raw--twinke_twinkle_3', 'raw--twinke_twinkle_2', 'raw--twinke_twinkle_27', 'raw--twinke_twinkle_22', 'raw--twinke_twinkle_12', 'raw--twinke_twinkle_20', 'raw--twinke_twinkle_13', 'raw--twinke_twinkle_9', 'raw--twinke_twinkle_18', 'raw--twinke_twinkle_23', 'raw--twinke_twinkle_15', 'raw--twinke_twinkle_21', 'raw--twinke_twinkle_25', 'raw--twinke_twinkle_14']


Some weights of VivitModel were not initialized from the model checkpoint at google/vivit-b-16x2-kinetics400 and are newly initialized: ['vivit.pooler.dense.bias', 'vivit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'BATCH_SIZE': 5, 'DEVICE': 'cuda', 'EPOCHS': 100, 'EXPERIMENT': 'full_twinkle_bsz5_lr0.0001', 'LOG_EVERY_X_EPOCHS': 1, 'LR': 0.0001, 'SEED': 43, 'USE_WANDB': True, 'WANDB_ENTITY': 'elles', 'WANDB_GROUP': 'full_twinkle_bsz5_lr0.0001', 'WANDB_PROJECT': 'video2music', 'vocab_size': 328, 'hidden_size': 768, 'encoder_layers': 12, 'encoder_ffn_dim': 3072, 'encoder_attention_heads': 12, 'encoder_layerdrop': 0.1, 'decoder_layers': 6, 'decoder_ffn_dim': 3072, 'decoder_attention_heads': 12, 'decoder_layerdrop': 0.1, 'hidden_act': 'gelu', 'positional_dropout': 0.1, 'hidden_dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 0.1, 'initializer_range': 0.02, 'layer_norm_eps': 1e-05, 'scale_embedding': False, 'feat_extract_norm': 'group', 'feat_proj_dropout': 0.0, 'feat_extract_activation': 'gelu', 'conv_dim': [512, 512, 512, 512, 512, 512, 512], 'conv_stride': [5, 2, 2, 2, 2, 2, 2], 'conv_kernel': [10, 3, 3, 3, 3, 2, 2], 'conv_bias': False, 'num_conv_pos_embeddings': 128, 'num_conv_pos_e

batch_accuracy,▁▃▃▃▅▄▆▆▆▆▆▆▆▆█▆▆▆
epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
epoch_avg_loss,█▅▄▃▃▃▃▂▂▂▂▂▂▂▁▂▁▁
batch_accuracy,0.5625
epoch,17
epoch_avg_loss,1.74242


...

Epoch 99/100: 100%|██████████| 15/15 [01:01<00:00,  4.10s/it, loss=0.0364]


{'epoch': 98, 'epoch_avg_loss': 0.38644389708836874, 'batch_accuracy': 1.0}


Epoch 100/100: 100%|██████████| 15/15 [01:00<00:00,  4.02s/it, loss=0.149]


{'epoch': 99, 'epoch_avg_loss': 0.5174178719520569, 'batch_accuracy': 1.0}


batch_accuracy,▁▃▃▅▅▅▅▅▆▅▇▇▇▇▇▇▇▇▇▇████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_avg_loss,█▆▅▅▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁
batch_accuracy,1.0
epoch,99
epoch_avg_loss,0.51742


In [15]:
model.eval()
with torch.no_grad():
  inputs = {
        "encoder_outputs": last_hidden_states.to('cuda'),
        "decoder_input_ids": labels.unsqueeze(0)
        # "attention_mask": batch["attention_mask"].to('cuda'),
        # "labels": batch["decoder_input_ids"].to('cuda'), # TODO: this for sure gets shifted automatically by the library no?
  }
  outputs = model(**inputs)

# Check that the trained model is outputting correct sequence

In [21]:
compute_accuracy(outputs, labels)

1.0

In [17]:
  logits = outputs.logits
  predicted_ids = torch.argmax(logits, dim=-1)

In [18]:
predicted_ids

tensor([[  4, 189, 308,  32, 109, 140,  44, 110, 134, 199,  44, 111, 130]],
       device='cuda:0')

In [19]:
labels

tensor([  4, 189, 308,  32, 109, 140,  44, 110, 134, 199,  44, 111, 130],
       device='cuda:0')

In [ ]:
# tokens = midi_tokens[batch_i]
# tokenss.append(tokens)
# midi = tokenizer([tokens])
# midis.append(midi)
# midi_translation = [token_id_to_token[int(id_.detach().numpy())] for id_ in tokens]
# midi_translations.append(midi_translation)